In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload

In [3]:
import drivers, lib
from time import sleep
import pickle

In [4]:
from lib import data_management as dm

In [5]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import E8257D

In [6]:
exa = Agilent_EXA_N9010A("SA", "EXA")
awg = KeysightAWG("AWG2")
mwsrc = E8257D("MXG")

In [8]:
# reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

In [18]:
cal = IQCalibrator(awg, exa, mwsrc, "TEST", -20)
res = cal.calibrate(lo_frequency=7e9, if_frequency=100e6, lo_power=10,
                    ssb_power=-20, iterations=1, minimize_iterlimit=20,sa_res_bandwidth=1000,
                    initial_guess=res.get_optimization_results()[0])

Phase:  [ 1.21827332] [-19.982526779174805, -72.50257873535156, -97.51728057861328], 

In [12]:
print(res)

Calibration data for mixer TEST
Mixer parameters: {'iq_attenuation': -20, 'mixer_id': 'TEST'}
Radiation parameters: {'lo_frequency': 7000000000.0, 'lo_power': 10, 'if_frequency': 100000000.0, 'ssb_power': -20}
Optimization results: {'if': [-20.63229751586914, -66.41133880615234, -83.86714935302734], 'dc': -89.488754272460938}
Optimization time: 0 h 0 m 31.7 s


In [19]:
data = res.get_optimization_results()[0]

In [20]:
data

{'dc_offsets': array([-0.21647701,  0.06030723]),
 'if_amplitudes': array([ 1.0493314 ,  1.06509741]),
 'if_offsets': array([-0.20835912,  0.05540727]),
 'if_phase': array([ 3.82731852])}

In [16]:
data

{'dc_offsets': array([-0.21647701,  0.06030723]),
 'if_amplitudes': array([ 0.97305017,  0.98762395]),
 'if_offsets': array([-0.20718583,  0.06392773]),
 'if_phase': array([ 3.9579622])}

In [17]:
awg.output_continuous_wave(frequency=100e6, amplitude=data["if_amplitudes"][0], phase=data["if_phase"],
                           offset=data["dc_offsets"][0], channel=1)
awg.output_continuous_wave(frequency=100e6, amplitude=data["if_amplitudes"][1], phase=0,
                           offset=data["dc_offsets"][1], channel=2)

In [21]:
exa.setup_swept_sa(7e9, 250e6, nop=1001, rbw=1e4)
exa.set_continuous()

In [ ]:
reload(dm)

In [22]:
dm.save_IQMX_calibration(res)

In [ ]:
res.get_radiation_parameters()

In [ ]:
list(dm.load_IQMX_calibration_database("TEST", -20).values())[0].get_optimization_results()

In [ ]:
list(dm.load_IQMX_calibration_database("TEST", -20).keys())

In [ ]:
exa.sweep_single()
exa.get_tracedata()